In [1]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.10.9 (main, Dec 15 2022, 17:11:09) [Clang 14.0.0 (clang-1400.0.29.202)], sys.version_info(major=3, minor=10, micro=9, releaselevel='final', serial=0) 
Pytorch version: 1.13.1 


/Users/carlschader/programming/cardscanner/tcg-yolov7/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!python detect.py --weights ./yolo.pt --conf 0.75 --img-size 640 --source ~/Desktop/joe/11.jpeg

Namespace(weights=['./yolo.pt'], source='/Users/carlschader/Desktop/joe/11.jpeg', img_size=640, conf_thres=0.75, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 ed7b162 torch 1.13.1 CPU

Fusing layers... 
IDetect.fuse
/Users/carlschader/anaconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

7 yugiohs, Done. (74.0ms) Inference, (9.0ms) NMS
 The image with the result 

In [6]:
!python export.py --weights ./yolo.pt --img-size 640 640

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='./yolo.pt', img_size=[640, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=False, end2end=False, max_wh=None, topk_all=100, iou_thres=0.45, conf_thres=0.25, device='cpu', simplify=False, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 ed7b162 torch 1.13.1 CPU

Fusing layers... 
IDetect.fuse
/Users/carlschader/anaconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS

Starting TorchScript export with torch 1.13.1...
TorchScript export success, saved as ./yolo.torchscript.pt
Torch version 1.13.1 has not been tested with coremlt

In [7]:
# Load Coreml-model
import coremltools as ct

model_filename = 'yolo.mlmodel'
model = ct.models.MLModel(model_filename)

Torch version 1.13.1 has not been tested with coremltools. You may run into unexpected errors. Torch 1.12.1 is the most recent version that has been tested.


In [8]:
# Load image
from PIL import Image
import cv2
import numpy as np

im = cv2.imread('/Users/carlschader/Desktop/joe/11.jpeg')
im = cv2.resize(im, (640, 640))
print(f" im = {im.shape}")
b = 1
h, w, ch = im.shape

im = Image.fromarray((im).astype('uint8'))
print(f" im = {im}")

 im = (640, 640, 3)
 im = <PIL.Image.Image image mode=RGB size=640x640 at 0x1069C3CA0>


In [9]:
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

In [10]:
# Inference only for MacOS and iOS!!!

y = model.predict({'image': im})  # coordinates are xywh normalized
if 'confidence' in y:
 box = xywh2xyxy(y['coordinates'] * [[w, h, w, h]])  # xyxy pixels
 conf, cls = y['confidence'].max(1), y['confidence'].argmax(1).astype(np.float)
 y = np.concatenate((box, conf.reshape(-1, 1), cls.reshape(-1, 1)), 1)
else:
 k = 'var_' + str(sorted(int(k.replace('var_', '')) for k in y)[-1])  # output key
 y = y[k]  # output

print(y)

[[[[[ 2.85888672e-01  9.93164062e-01 -7.21679688e-01 -2.65380859e-01
     -7.70703125e+00  3.90039062e+00]
    [-2.61474609e-01  4.84619141e-02 -4.86572266e-01 -6.74316406e-01
     -7.92578125e+00  3.92187500e+00]
    [ 4.93408203e-01 -1.74926758e-01  1.38549805e-02 -8.50097656e-01
     -8.17968750e+00  3.99414062e+00]
    ...
    [-3.51562500e-01  8.69750977e-02  1.22131348e-01 -5.71289062e-01
     -1.14140625e+01  3.98632812e+00]
    [-4.35546875e-01  1.50024414e-01 -2.16430664e-01 -5.83984375e-01
     -1.00937500e+01  4.17968750e+00]
    [-1.10717773e-01  5.50781250e-01 -4.96093750e-01 -3.11035156e-01
     -9.13281250e+00  4.08593750e+00]]

   [[ 1.34155273e-01  8.17871094e-01 -8.64257812e-01  3.92089844e-01
     -7.86718750e+00  3.90429688e+00]
    [-9.87792969e-01  1.39282227e-01 -6.13769531e-01  8.78906250e-02
     -8.82812500e+00  4.16406250e+00]
    [ 1.12792969e+00  1.04309082e-01  1.42089844e-01  2.38037109e-03
     -1.21953125e+01  3.99023438e+00]
    ...
    [-1.00390625e+0